# Import statements

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='/home/ubuntu/fastai/mercari/data/'

In [3]:
!ls data

sample_submission.csv  test.tsv  train.tsv


In [4]:
train = pd.read_csv(f'{PATH}train.tsv', sep='\t')

In [5]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
test = pd.read_csv(f'{PATH}test.tsv', sep='\t')
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


# Feature creation

In [7]:
cat_vars = ['name', 'item_condition_id', 'category_name', 'brand_name', 'shipping', 'item_description']
contin_vars = []

In [8]:
dep = 'price'
test[dep] = 0
joined = train[cat_vars+[dep]].copy()
joined_test = test[cat_vars+[dep,  'test_id']].copy()

In [ ]:
#joined.head()

In [ ]:
#DataFrameSummary(joined).summary()

In [ ]:
#joined_test.head()

In [9]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [10]:
apply_cats(joined_test, joined)

In [11]:
for v in contin_vars:
    joined[v] = joined[v].astype('float32')
    joined_test[v] = joined_test[v].astype('float32')

In [12]:
joined['item_condition_id'] = joined.item_condition_id.astype(np.int32)
joined['shipping'] = joined.shipping.astype(np.int32)
joined_test['item_condition_id'] = joined_test.item_condition_id.astype(np.int32)
joined_test['shipping'] = joined_test.shipping.astype(np.int32)

In [13]:
n = len(joined)
idxs = get_cv_idxs(n, val_pct=150000/n)
joined_samp = joined.iloc[idxs].set_index("name")
samp_size = len(joined_samp); samp_size

150000

In [20]:
??get_cv_idxs

In [14]:
samp_size = n
joined_samp = joined.set_index("name")

In [15]:
joined_samp.head()

,item_condition_id,category_name,brand_name,shipping,item_description,price
name,,,,,,
MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,1,No description yet,10.0
Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,0,This keyboard is in great condition and works ...,52.0
AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,1,Adorable top with a hint of lace and a key hol...,10.0
Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,1,New with tags. Leather horses. Retail for [rm]...,35.0
24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,0,Complete with certificate of authenticity,44.0


In [16]:
#??proc_df

In [17]:
len(joined_samp)

1482535

In [18]:
joined_test = joined_test.set_index("name")

In [21]:
df, y, nas, mapper = proc_df(joined_samp, 'price', do_scale=True)
yl = np.log(y)

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [22]:
df_test, _, nas, mapper = proc_df(joined_test, 'price', do_scale=True, skip_flds=['test_id'],
                                  mapper=mapper, na_dict=nas)
#for n,c in joined_samp.items(): print(n), print(c)

In [23]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [ ]:
np.unique()

In [28]:
len(val_idx)

370634

In [27]:
val_idx

[1111901,
 1111902,
 1111903,
 1111904,
 1111905,
 1111906,
 1111907,
 1111908,
 1111909,
 1111910,
 1111911,
 1111912,
 1111913,
 1111914,
 1111915,
 1111916,
 1111917,
 1111918,
 1111919,
 1111920,
 1111921,
 1111922,
 1111923,
 1111924,
 1111925,
 1111926,
 1111927,
 1111928,
 1111929,
 1111930,
 1111931,
 1111932,
 1111933,
 1111934,
 1111935,
 1111936,
 1111937,
 1111938,
 1111939,
 1111940,
 1111941,
 1111942,
 1111943,
 1111944,
 1111945,
 1111946,
 1111947,
 1111948,
 1111949,
 1111950,
 1111951,
 1111952,
 1111953,
 1111954,
 1111955,
 1111956,
 1111957,
 1111958,
 1111959,
 1111960,
 1111961,
 1111962,
 1111963,
 1111964,
 1111965,
 1111966,
 1111967,
 1111968,
 1111969,
 1111970,
 1111971,
 1111972,
 1111973,
 1111974,
 1111975,
 1111976,
 1111977,
 1111978,
 1111979,
 1111980,
 1111981,
 1111982,
 1111983,
 1111984,
 1111985,
 1111986,
 1111987,
 1111988,
 1111989,
 1111990,
 1111991,
 1111992,
 1111993,
 1111994,
 1111995,
 1111996,
 1111997,
 1111998,
 1111999,
 1112000,


In [29]:
df.head()

,item_condition_id,category_name,brand_name,shipping,item_description
name,,,,,
MLB Cincinnati Reds T Shirt Size XL,1.209777,809,0,1.111649,806611
Razer BlackWidow Chroma Keyboard,1.209777,87,3558,-0.899564,1090879
AVA-VIV Blouse,-1.004675,1255,4181,1.111649,115290
Leather Horse Statues,-1.004675,486,0,1.111649,782306
24K GOLD plated rose,-1.004675,1182,0,-0.899564,366653


In [ ]:
len(joined)

In [26]:
df_test.head()

,item_condition_id,category_name,brand_name,shipping,item_description
name,,,,,
NaN,-1.004675,1183,0,1.111649,960367
"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",-1.004675,878,0,1.111649,66133
Coach bag,-1.004675,978,1010,1.111649,0
Floral Kimono,0.102551,1234,0,-0.899564,0
NaN,1.209777,840,0,1.111649,0


In [24]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [25]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl, cat_flds=cat_vars, bs=128, 
                                       test_df=df_test)

KeyError: "['name'] not in index"